# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.4 *k*-NNや最小二乗法による回帰分析と回帰モデルの推定性能の評価

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの分子記述子 (特徴量) で化学構造が数値化されています。特徴量は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
dataset.shape

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を 説明変数 x とします

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

In [ ]:
y # 念のため確認

## k-NN, OLS による回帰分析の実行

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

下はテキスト化していますが、shuffle=False とすると、ランダムに分割されるのではなく、下から test_size の数のサンプルがテストデータに、残りのサンプルがトレーニングデータになります。時系列データにおいて、時間的に古いサンプルをトレーニングデータに、新しいサンプルをテストデータとしたいときなどに利用します。

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=False)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

k-NN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
k_in_knn = 5 # k の値

In [ ]:
model = KNeighborsRegressor(n_neighbors=k_in_knn, metric='euclidean')

`metric` の種類 (第 7 章のクラスタリングのときの `metric` とは異なることに注意してください)
- 'euclidean' : ユークリッド距離
- 'manhattan' : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) をご覧ください 

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータの y の値の推定

In [ ]:
model.predict(autoscaled_x_train) #トレーニングデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # y のスケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

OLS

In [ ]:
from sklearn.linear_model import LinearRegression # OLS モデル構築やモデルを用いた y の値の推定に使用

In [ ]:
model = LinearRegression()

上の二つのセルは、k-NN と OLS とで異なりますが、以降のセルにおいて OLS で標準回帰係数を確認・保存する部分以外は、k-NN と OLS とで全く同じです。scikit-learn を活用することで、いろいろな手法を同じ形式で実行できます。

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築。OLS では、標準回帰係数を計算することに対応します

トレーニングデータの y の値の推定

In [ ]:
model.predict(autoscaled_x_train) #トレーニングデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータの y の値を推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

OLS モデルの標準回帰係数

In [ ]:
model.coef_ # 標準回帰係数。array 型で出力されます

In [ ]:
standard_regression_coefficients = pd.DataFrame(model.coef_) # pandas の DataFrame 型に変換

In [ ]:
standard_regression_coefficients.index = x_train.columns # 説明変数に対応する名前を、元のデータセットにおける説明変数の名前に

In [ ]:
standard_regression_coefficients.columns = ['standard_regression_coefficients'] # 列名を変更

In [ ]:
standard_regression_coefficients # 念のため確認

In [ ]:
standard_regression_coefficients.to_csv('standard_regression_coefficients.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

例えば HeavyAtomMolWt (水素原子以外で計算された分子量) の標準回帰係数が負に大きい値となっています。水素原子以外で計算された分子量の大きな化合物は沸点が低く推定されるということです。分子量の大きな化合物は沸点が高い傾向がある、という知見と一致しません。この標準回帰係数に関する考察は次回に詳しく行います。

k-NN と OLS とで、回帰モデルの推定性能を比較してみましょう。k-NN では、k の値を変えてそれぞれの回帰モデルの推定性能を評価してみましょう。

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

#### 練習問題
仮想的な樹脂材料のデータセット用いて、kNN や OLS により回帰モデルを構築し、y が未知のデータセットの推定を行いましょう。

原料として 3 種類 (raw material 1, raw material 2, raw material 3) あり、それらの組成比と重合温度 (temperature)・重合時間 (time) をそれぞれ変えて樹脂材料が作られ、物性 a (property a)と物性 b (property b) が測定されたような 20 サンプルが、トレーニングデータとしてあるとします。物性 a・物性 b ともに、大きいほど樹脂材料として望ましいとします。回帰分析においては、物性 a と物性 b が目的変数であり、3 つの原料の組成比・重合温度・重合時間の 5 つの特徴量が説明変数です。データセットのファイルは virtual_resin.csv です。まず、このファイルを用いて kNN や OLS により回帰モデルを構築しましょう。次に、virtual_resin_prediction.csv にある、まだ実験していない 8164 通りの組成比・重合温度・重合時間において、構築した kNN モデル や OLS モデルで物性 a, 物性 b の値を推定しましょう。

コードの例は下にあります。

#### 練習問題 コードの例

k-NN の場合

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('virtual_resin.csv', index_col=0, header=0) # 仮想的な樹脂のデータセットの読み込み

In [ ]:
dataset.shape

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 2:] # 説明変数 x

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0:2] # 目的変数 y (2つ)

In [ ]:
y # 念のため確認

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_y = (y - y.mean()) / y.std() # トレーニングデータの目的変数の標準化

k-NN モデルの構築

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
k_in_knn = 5 # k の値

In [ ]:
model_1_knn = KNeighborsRegressor(n_neighbors=k_in_knn, metric='euclidean') # property_a 用

In [ ]:
model_2_knn = KNeighborsRegressor(n_neighbors=k_in_knn, metric='euclidean') # property_b 用

In [ ]:
model_1_knn.fit(autoscaled_x, autoscaled_y.iloc[:, 0]) # property_a の回帰モデルの構築

In [ ]:
model_2_knn.fit(autoscaled_x, autoscaled_y.iloc[:, 1]) # property_b の回帰モデルの構築

k-NN モデルによるトレーニングデータの y の値の推定

property_a

In [ ]:
estimated_y_1_knn = pd.DataFrame(model_1_knn.predict(autoscaled_x))

In [ ]:
estimated_y_1_knn = estimated_y_1_knn * y.iloc[:, 0].std() + y.iloc[:, 0].mean() # y のスケールをもとに戻します

In [ ]:
estimated_y_1_knn.index = x.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_1_knn.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_1_knn # 念のため確認

In [ ]:
estimated_y_1_knn.to_csv('estimated_y_1_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_b

In [ ]:
estimated_y_2_knn = pd.DataFrame(model_2_knn.predict(autoscaled_x))

In [ ]:
estimated_y_2_knn = estimated_y_2_knn * y.iloc[:, 1].std() + y.iloc[:, 1].mean() # y のスケールをもとに戻します

In [ ]:
estimated_y_2_knn.index = x.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_2_knn.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_2_knn # 念のため確認

In [ ]:
estimated_y_2_knn.to_csv('estimated_y_2_knn.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

property_a

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y.iloc[:, 0], estimated_y_1_knn.iloc[:, 0]) # 散布図
y_max = max(y.iloc[:, 0].max(), estimated_y_1_knn.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.iloc[:, 0].min(), estimated_y_1_knn.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

property_b

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y.iloc[:, 1], estimated_y_2_knn.iloc[:, 0]) # 散布図
y_max = max(y.iloc[:, 1].max(), estimated_y_2_knn.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.iloc[:, 1].min(), estimated_y_2_knn.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

property_a

In [ ]:
metrics.r2_score(y.iloc[:, 0], estimated_y_1_knn) # r2

In [ ]:
metrics.mean_absolute_error(y.iloc[:, 0], estimated_y_1_knn) # MAE

property_b

In [ ]:
metrics.r2_score(y.iloc[:, 1], estimated_y_2_knn) # r2

In [ ]:
metrics.mean_absolute_error(y.iloc[:, 1], estimated_y_2_knn) # MAE

y が未知のサンプルを用いた、y の値の推定

In [ ]:
x_prediction = pd.read_csv('virtual_resin_prediction.csv', encoding='SHIFT-JIS', index_col=0)  # y が未知のデータセットの読み込み

In [ ]:
x_prediction  # 念のため確認

オートスケーリング

In [ ]:
autoscaled_x_prediction = (x_prediction - x.mean()) / x.std()

property_a

In [ ]:
estimated_y_1_knn_prediction = pd.DataFrame(model_1_knn.predict(autoscaled_x_prediction)) # 推定後、pandas の DataFrame 型に変換

In [ ]:
estimated_y_1_knn_prediction = estimated_y_1_knn_prediction * y.iloc[:, 0].std() + y.iloc[:, 0].mean() # スケールをもとに戻します

In [ ]:
estimated_y_1_knn_prediction.index = x_prediction.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_1_knn_prediction.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_1_knn_prediction # 念のため確認

In [ ]:
estimated_y_1_knn_prediction.to_csv('estimated_y_1_knn_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_b

In [ ]:
estimated_y_2_knn_prediction = pd.DataFrame(model_2_knn.predict(autoscaled_x_prediction)) # 推定後、pandas の DataFrame 型に変換

In [ ]:
estimated_y_2_knn_prediction = estimated_y_2_knn_prediction * y.iloc[:, 1].std() + y.iloc[:, 1].mean() # スケールをもとに戻します

In [ ]:
estimated_y_2_knn_prediction.index = x_prediction.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_2_knn_prediction.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_2_knn_prediction # 念のため確認

In [ ]:
estimated_y_2_knn_prediction.to_csv('estimated_y_2_knn_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_a vs. property_b プロット (青点が y 未知のサンプル、赤点がトレーニングデータのサンプル)

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(estimated_y_1_knn_prediction.iloc[:, 0], estimated_y_2_knn_prediction.iloc[:, 0], c='blue') # 散布図
plt.scatter(y.iloc[:, 0], y.iloc[:, 1], c='red') # トレーニングデータの散布図
plt.xlabel("property_a") # x 軸の名前
plt.ylabel("property_b") # y 軸の名前
plt.show() # 以上の設定で描画

OLS の場合

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('virtual_resin.csv', index_col=0) # 仮想的な樹脂のデータセットの読み込み

In [ ]:
dataset.shape

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 2:] # 説明変数 x

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0:2] # 目的変数 y (2つ)

In [ ]:
y # 念のため確認

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_y = (y - y.mean()) / y.std() # トレーニングデータの目的変数の標準化

OLS モデルの構築

In [ ]:
from sklearn.linear_model import LinearRegression # OLS モデル構築やモデルを用いた y の値の推定に使用

In [ ]:
model_1_ols = LinearRegression() # property_a 用

In [ ]:
model_2_ols = LinearRegression() # property_b 用

In [ ]:
model_1_ols.fit(autoscaled_x, autoscaled_y.iloc[:, 0]) # property_a の回帰モデルの構築

In [ ]:
model_2_ols.fit(autoscaled_x, autoscaled_y.iloc[:, 1]) # property_b の回帰モデルの構築

OLS モデルによるトレーニングデータの y の値の推定

property_a

In [ ]:
estimated_y_1_ols = pd.DataFrame(model_1_ols.predict(autoscaled_x))

In [ ]:
estimated_y_1_ols = estimated_y_1_ols * y.iloc[:, 0].std() + y.iloc[:, 0].mean() # y のスケールをもとに戻します

In [ ]:
estimated_y_1_ols.index = x.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_1_ols.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_1_ols # 念のため確認

In [ ]:
estimated_y_1_ols.to_csv('estimated_y_1_ols.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_b

In [ ]:
estimated_y_2_ols = pd.DataFrame(model_2_ols.predict(autoscaled_x))

In [ ]:
estimated_y_2_ols = estimated_y_2_ols * y.iloc[:, 1].std() + y.iloc[:, 1].mean() # y のスケールをもとに戻します

In [ ]:
estimated_y_2_ols.index = x.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_2_ols.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_2_ols # 念のため確認

In [ ]:
estimated_y_2_ols.to_csv('estimated_y_2_ols.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

property_a

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y.iloc[:, 0], estimated_y_1_ols.iloc[:, 0]) # 散布図
y_max = max(y.iloc[:, 0].max(), estimated_y_1_ols.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.iloc[:, 0].min(), estimated_y_1_ols.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

property_b

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y.iloc[:, 1], estimated_y_2_ols.iloc[:, 0]) # 散布図
y_max = max(y.iloc[:, 1].max(), estimated_y_2_ols.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.iloc[:, 1].min(), estimated_y_2_ols.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

property_a

In [ ]:
metrics.r2_score(y.iloc[:, 0], estimated_y_1_ols) # r2

In [ ]:
metrics.mean_absolute_error(y.iloc[:, 0], estimated_y_1_ols) # MAE

property_b

In [ ]:
metrics.r2_score(y.iloc[:, 1], estimated_y_2_ols) # r2

In [ ]:
metrics.mean_absolute_error(y.iloc[:, 1], estimated_y_2_ols) # MAE

y が未知のサンプルを用いた、y の値の推定

In [ ]:
x_prediction = pd.read_csv('virtual_resin_prediction.csv', encoding='SHIFT-JIS', index_col=0)  # y が未知のデータセットの読み込み

In [ ]:
x_prediction  # 念のため確認

オートスケーリング

In [ ]:
autoscaled_x_prediction = (x_prediction - x.mean()) / x.std()

property_a

In [ ]:
estimated_y_1_ols_prediction = pd.DataFrame(model_1_ols.predict(autoscaled_x_prediction)) # 推定後、pandas の DataFrame 型に変換

In [ ]:
estimated_y_1_ols_prediction = estimated_y_1_ols_prediction * y.iloc[:, 0].std() + y.iloc[:, 0].mean() # スケールをもとに戻します

In [ ]:
estimated_y_1_ols_prediction.index = x_prediction.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_1_ols_prediction.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_1_ols_prediction # 念のため確認

In [ ]:
estimated_y_1_ols_prediction.to_csv('estimated_y_1_ols_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_b

In [ ]:
estimated_y_2_ols_prediction = pd.DataFrame(model_2_ols.predict(autoscaled_x_prediction)) # 推定後、pandas の DataFrame 型に変換

In [ ]:
estimated_y_2_ols_prediction = estimated_y_2_ols_prediction * y.iloc[:, 1].std() + y.iloc[:, 1].mean() # スケールをもとに戻します

In [ ]:
estimated_y_2_ols_prediction.index = x_prediction.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_2_ols_prediction.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_2_ols_prediction # 念のため確認

In [ ]:
estimated_y_2_ols_prediction.to_csv('estimated_y_2_ols_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

property_a vs. property_b プロット (青点が y 未知のサンプル、赤点がトレーニングデータのサンプル)

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(estimated_y_1_ols_prediction.iloc[:, 0], estimated_y_2_ols_prediction.iloc[:, 0], c='blue') # 散布図
plt.scatter(y.iloc[:, 0], y.iloc[:, 1], c='red') # トレーニングデータの散布図
plt.xlabel("property_a") # x 軸の名前
plt.ylabel("property_b") # y 軸の名前
plt.show() # 以上の設定で描画